In [106]:
import numpy as np
import pandas as pd 
# import holoviews as hv
# from holoviews import dim, opts
# import bokeh.io
# import iqplot
import urllib
import re
import helper
import random

In [107]:
# Run this to reload helper.py so you don't have to restart the kernel
import importlib

importlib.reload(helper)

<module 'helper' from '/Users/prakash/Desktop/Courses/ML/loan-sharks/poems/helper.py'>

## Preprocess Data

In [108]:
all_words, all_sequences, word_dict, all_sonnet_int, syllable_dict= helper.getAllWordsAndSequences("data/shakespeare.txt", "data/Syllable_dictionary.txt",syllable_count=True)

In [109]:
all_sonnet_int[11]

[1522, 2210, 457, 2683, 1506, 2114, 609, 3205]

## Utility function

In [110]:
def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r

## Training HMM

In [111]:
for idx in range(len(all_sonnet_int)):
    if len(all_sonnet_int[idx])>1:
        new_sonet_sequence.append(all_sonnet_int[idx][-2::-1])

In [112]:
hmm5 = helper.unsupervised_HMM(new_sonet_sequence, 30, 100)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100


## Rhyme

In [113]:
rhyme=[]
rhyme_couplet=[]
new_sonet_sequence=[]
for idx in range(len(all_sonnet_int)):
    if (idx+2)%14==0 and len(all_sonnet_int[idx+1])>1:
        rhyme_couplet.append((all_sonnet_int[idx][-2],all_sonnet_int[idx+1][-2]))
    elif idx%4==0 and len(all_sonnet_int[idx+2])>1:
        rhyme.append((all_sonnet_int[idx][-2],all_sonnet_int[idx+2][-2]))
        rhyme.append((all_sonnet_int[idx+1][-2],all_sonnet_int[idx+3][-2]))

In [114]:
def sample_rev_sonents(hmm, obs_map, syllable_dict, n_sonnet=10, start_word=None):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)
    start_state = None
    if start_word:
        emition_pos=[hmm.O[i][start_word] for i in range(hmm.L)]
        start_state=random.choices([i for i in range(hmm.L)],weights=emition_pos)[0]
    # Sample and convert sentence.
    emission = []
    states = []
    count_sonet = int(syllable_dict[obs_map_r[start_word]][-1])
    state=start_state
#     print(obs_map_r[start_word])

    for i in range(100):
            # Append state.
            states.append(state)
            while(True):

                # Sample next observation.
                rand_var = random.uniform(0, 1)
                next_obs = 0

                while rand_var > 0:
                    rand_var -= hmm.O[state][next_obs]
                    next_obs += 1

                next_obs -= 1
                emission.append(next_obs)
                if i!=0:
                    count_sonet+=int(syllable_dict[obs_map_r[next_obs]][-1])
                if count_sonet==10:
                    break
                elif count_sonet>10:
                    count_sonet-=int(syllable_dict[obs_map_r[next_obs]][-1])
                    emission.pop()
                    continue
                else:
                    break
            if count_sonet==10:
                break

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0

            while rand_var > 0:
                rand_var -= hmm.A[state][next_state]
                next_state += 1

            next_state -= 1
            state = next_state
    sentence = [obs_map_r[i] for i in emission]
    
    if start_word:
        sentence[0]=obs_map_r[start_word]

    return ' '.join(sentence[::-1]).capitalize()

In [115]:
print('Sample Sonnet:\n====================')
sentence_rhyme=['' for i in range(14)]
mapper=obs_map_reverser(word_dict)
for i in range(3):
    new_rhyme_1=random.choice(rhyme)
    new_rhyme_2=random.choice(rhyme)
    sentence_rhyme[i*4]=new_rhyme_1[0]
    sentence_rhyme[i*4+1]=new_rhyme_2[0]
    sentence_rhyme[i*4+2]=new_rhyme_1[1]
    sentence_rhyme[i*4+3]=new_rhyme_2[1]
#     print(new_rhyme_1,new_rhyme_2)
new_rhyme_1=random.choice(rhyme_couplet)
sentence_rhyme[-2]=new_rhyme_1[0]
sentence_rhyme[-1]=new_rhyme_1[1]
# print(mapper[new_rhyme_1[0]],mapper[new_rhyme_1[1]])
for start_rhyme in sentence_rhyme:
    print(sample_rev_sonents(hmm5, word_dict, syllable_dict, n_sonnet=10, start_word=start_rhyme))

Sample Sonnet:
Earth when in show the a powerful confound
Conscience in this and pays a learned's pain
Best still darkening the joy unless i'll brow
Ten yet in music looks shall with my east
Bending with her owner's due that your view
Were would to sweet'st gone that your winter's trust
Do name thief thy image with o in new
Mayst in power lilies heavy with of me
To be distilled and all that is that face
Away of brain lawful end string ride missed
Little fair muse for so am breed so place
Pain and will foul be what expense of score
Truth to 'gainst doth face the of of worth gone
Theirs which me sleep in new thou be alone


## Meter

In [18]:
import nltk
nltk.download('cmudict')

[nltk_data] Downloading package cmudict to /Users/prakash/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


True

In [116]:
from nltk.corpus import cmudict

In [117]:
def stress_syllable(word,prondict):
    start,end=-1,-1
    if word not in prondict:
        return start,end
    processed=[ele[-1] for ele in prondict[word][0]]
    for ele in processed:
        if ele.isnumeric():
            start=int(ele)
            break
    for ele in processed[::-1]:
        if ele.isnumeric():
            end=int(ele)
            break
    return start,end

In [118]:
prondict = cmudict.dict()
stress_syllable('compare',prondict)

(0, 1)

In [119]:
def sample_rev_sonents_meter(hmm, obs_map, syllable_dict, prondict, n_sonnet=10, start_word=None):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)
    start_state = None
    if start_word:
        emition_pos=[hmm.O[i][start_word] for i in range(hmm.L)]
        start_state=random.choices([i for i in range(hmm.L)],weights=emition_pos)[0]
    # Sample and convert sentence.
    emission = []
    states = []
    count_sonet = int(syllable_dict[obs_map_r[start_word]][-1])
    start,_ = stress_syllable(obs_map_r[start_word],prondict)
    state=start_state
#     print(obs_map_r[start_word])

    for i in range(100):
            # Append state.
            states.append(state)
            while(True):

                # Sample next observation.
                rand_var = random.uniform(0, 1)
                next_obs = 0

                while rand_var > 0:
                    rand_var -= hmm.O[state][next_obs]
                    next_obs += 1

                next_obs -= 1
                start_new,end_new = stress_syllable(obs_map_r[next_obs],prondict)
                if end_new!=start:
                    continue
                start=start_new
                emission.append(next_obs)
                if i!=0:
                    count_sonet+=int(syllable_dict[obs_map_r[next_obs]][-1])
                if count_sonet==10:
                    break
                elif count_sonet>10:
                    count_sonet-=int(syllable_dict[obs_map_r[next_obs]][-1])
                    emission.pop()
                    continue
                else:
                    break
            if count_sonet==10:
                break

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0

            while rand_var > 0:
                rand_var -= hmm.A[state][next_state]
                next_state += 1

            next_state -= 1
            state = next_state
    sentence = [obs_map_r[i] for i in emission]
    
    if start_word:
        sentence[0]=obs_map_r[start_word]

    return ' '.join(sentence[::-1]).capitalize()

In [204]:
print('Sample Sonnet:\n====================')
sentence_rhyme=['' for i in range(14)]
mapper=obs_map_reverser(word_dict)
for i in range(3):
    new_rhyme_1=random.choice(rhyme)
    new_rhyme_2=random.choice(rhyme)
    sentence_rhyme[i*4]=new_rhyme_1[0]
    sentence_rhyme[i*4+1]=new_rhyme_2[0]
    sentence_rhyme[i*4+2]=new_rhyme_1[1]
    sentence_rhyme[i*4+3]=new_rhyme_2[1]
#     print(new_rhyme_1,new_rhyme_2)
new_rhyme_1=random.choice(rhyme_couplet)
sentence_rhyme[-2]=new_rhyme_1[0]
sentence_rhyme[-1]=new_rhyme_1[1]
# print(mapper[new_rhyme_1[0]],mapper[new_rhyme_1[1]])
for start_rhyme in sentence_rhyme:
    print(sample_rev_sonents_meter(hmm5, word_dict, syllable_dict, prondict, n_sonnet=10, start_word=start_rhyme))

Sample Sonnet:
Sable towards eyes which thine knows of their name
Ills him their with my faults it old or true
Do my sword with my with weep to all age
Name was my this young that it indigest
Left o grew not that i men's to endure
My deeds or one that long be for of glass
Me should to times me when be such woe cure
Thy mind thy thanks of thy dear my love was
Love than heart full this nothing within grace
Hate did by thee his mind's that my love is
Grave lips this but love on my large my black
Of it are at from how be rest amis
Bosom disgrace of thy sweet i from one
Nor o love will i have seen fair alone


## Mixing Spenser